# Import libraries

In [1]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.1 MB 1.1 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.1 MB 1.8 MB/s eta 0:00:02
   -------------------- ------------------- 1.1/2.1 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-surprise

     ---------------------------------------- 0.0/772.0 kB ? eta -:--:--
      --------------------------------------- 10.2/772.0 kB ? eta -:--:--
     -- ---------------------------------- 61.4/772.0 kB 656.4 kB/s eta 0:00:02
     -------------- ----------------------- 286.7/772.0 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 772.0/772.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [101 lines of output]
  C:\Users\vidit\AppData\Local\Temp\pip-install-8hs1gutq\scikit-surprise_717076a5db234394b975fee029d50f24\setup.py:65: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!
  
          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************
  
  !!
    dist.Distribution().fetch_build_eggs(["numpy>=1.17.3"])
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\surprise
  copying surprise\accuracy.py -> build\lib.win-amd64-cpython-311\surprise
  copying surprise\bu

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
import gc
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets

In [28]:
# Load Movies.csv
movies = pd.read_csv("ml-20m/movies.csv")

# Load Ratings.csv
ratings = pd.read_csv("ml-20m/ratings.csv")

# Load Tags.csv
tags = pd.read_csv("ml-20m/tags.csv")

# Merge tags and movies

In [29]:
# collecting tags in movies dataframe for content
for j in tags.movieId.unique():
   movies.loc[movies.loc[movies.movieId==j].index,'tags']=' '.join([i if type(i)==str else str(i) for i in tags.loc[ ( tags.movieId == j ),'tag'].unique().tolist() ])

# Content filtering method

In [14]:
#splitting data for usability, larger datasets crashed kaggle and pc
from sklearn.model_selection import train_test_split
use,dontuse=train_test_split(ratings,test_size=0.995)

In [21]:
gc.collect()
user_movies_data=pd.pivot_table(use,index='movieId',columns='userId',values='rating',fill_value=0)
movies['tags']=movies['tags'].fillna('None')

In [22]:
tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(movies['tags'])
tfidf_df=pd.DataFrame(tfidf_matrix.toarray(),index=movies.index.tolist())

In [23]:
svd=TruncatedSVD(n_components=19)
latent_matrix=svd.fit_transform(tfidf_df)

In [24]:
n=19
latent_matrix_1_df=pd.DataFrame(latent_matrix[:,0:n],index=movies['title'].tolist())

# Create Collab filter

In [25]:
svd=TruncatedSVD(n_components=20)
latent_matrix_2=svd.fit_transform(user_movies_data)

In [26]:
latent_matrix_2_df=pd.DataFrame(latent_matrix_2,index=[movies.loc[(movies.movieId==i),'title'].values[0] for i in (use['movieId'].unique())])

In [27]:
#Finding issues with different sized matrixes
latent_matrix_1_df=latent_matrix_1_df.drop_duplicates()
latent_matrix_2_df=latent_matrix_2_df.drop_duplicates()

In [ ]:
a=latent_matrix_2_df.copy()
for i in latent_matrix_1_df.index:
    if i not in latent_matrix_2_df.index:
        a = pd.concat([a, pd.Series(np.zeros(20), index=np.arange(0, 20), name=i)])

        
b=latent_matrix_1_df.copy()
for i in a.index:
    if i not in b.index:
        b = pd.concat([b, pd.Series(np.zeros(19), index=np.arange(0, 19), name=i)])

        
#Same index was repeated multiple times, decided to remove duplicates
a = a[~a.index.duplicated(keep='first')]

b = b[~b.index.duplicated(keep='first')]

# Hybrid model

In [ ]:
def recommend_similar_movies(title):
    if title in b.index:
        a_1=np.array(b.loc[title]).reshape(1,-1)
        score_content=cosine_similarity(b,a_1).reshape(-1)
    else:
        score_content=0
    if title in a.index:
        a_2=np.array(a.loc[title]).reshape(1,-1)
        score_collab=cosine_similarity(a,a_2).reshape(-1)
    else:
        score_collab=0
    
    hybrid_score=(score_content+score_collab)/2

    dictDF={'content':score_content,'collab':score_collab,'hybrid':hybrid_score}
    dictDF
    similar_movies=pd.DataFrame(dictDF,index=a.index)


    similar_movies.sort_values('hybrid',ascending=False,inplace=True)
    return similar_movies

In [ ]:
recommend_similar_movies('Mission: Impossible II (2000)')